In [1]:
cd ..

/Users/mig/Desktop/pitt/projects/Group-Project-2140/cord_ir


In [90]:
from search.elastic_index_reader import IndexReader
from tqdm.notebook import trange, tqdm
import math

In [3]:
import xml.etree.ElementTree as ET
queryTree = ET.parse('../data/2020-07-16/eval/topics-rnd5.xml')

In [4]:
queryRoot = queryTree.getroot()
queries = []
for child in queryRoot:
    query = {
        'queryNo': child.attrib['number'],
        'query': child.find('query').text,
        'question': child.find('question').text,
        'narrative': child.find('narrative').text
    }
    queries.append(query)

In [5]:
reader = IndexReader()
for query in queries:
    query['query_tokens'] = [t['token'] for t in reader.tokenize(query['query'])['tokens']]
    query['question_tokens'] = [t['token'] for t in reader.tokenize(query['question'])['tokens']]
    query['narrative_tokens'] = [t['token'] for t in reader.tokenize(query['narrative'])['tokens']]

/opt/homebrew/Caskroom/miniforge/base/envs/info-retri/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [67]:
def getRetrievalResults(queries, field):
    results = {}
    for query in tqdm(queries):
        res = reader.search("cord_test", query[field], size=2000, fields=[], highlight=False)
        results[query['queryNo']] = res['hits']['hits']
    return results

In [68]:
queryFields = ['query', 'question', 'narrative']
results = {f: getRetrievalResults(queries, f) for f in queryFields}

  0%|          | 0/50 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/info-retri/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [69]:
# load the judgments
judgments = {q['queryNo']: [] for q in queries}
with open('../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt', 'r') as qrels:
    for line in qrels:
        [topicId, iteration, cordId, judgment] = line.strip('\n').split(' ')
        judgments[topicId].append({
            'iteration': iteration,
            'cordId': cordId,
            'judgment': judgment
        })

In [70]:
for field in queryFields:
    with open('../data/2020-07-16/eval/%s-result.txt'%field, 'w') as res_file:
        fieldResults = results[field]
        for query in tqdm(queries):
            for (index, result) in enumerate(fieldResults[query['queryNo']]):
                line = ' '.join([query['queryNo'], 'Q0', result['_id'], str(index + 1), str(result['_score']), field])
                res_file.write(line + '\n')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [73]:
# use trec_eval to get evaluation
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/query-result.txt
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/question-result.txt
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/narrative-result.txt

runid                 	all	query
num_q                 	all	50
num_ret               	all	100000
num_rel               	all	26664
num_rel_ret           	all	11040
map                   	all	0.1635
gm_map                	all	0.1021
Rprec                 	all	0.2523
bpref                 	all	0.3129
recip_rank            	all	0.8236
iprec_at_recall_0.00  	all	0.8466
iprec_at_recall_0.10  	all	0.4777
iprec_at_recall_0.20  	all	0.3524
iprec_at_recall_0.30  	all	0.2548
iprec_at_recall_0.40  	all	0.1607
iprec_at_recall_0.50  	all	0.0680
iprec_at_recall_0.60  	all	0.0160
iprec_at_recall_0.70  	all	0.0000
iprec_at_recall_0.80  	all	0.0000
iprec_at_recall_0.90  	all	0.0000
iprec_at_recall_1.00  	all	0.0000
P_5                   	all	0.6640
P_10                  	all	0.6180
P_15                  	all	0.6067
P_20                  	all	0.5840
P_30                  	all	0.5640
P_100                 	all	0.4620
P_200                 	all	0.3770
P_500                 	all	0.2539
P_1000               

In [130]:
# calculate in Python
ndcg_eval_at = [3, 5, 10, 15, 20, 25, 30]
recall_eval_at = [5, 10, 20, 50, 100, 300, 500, 1000, 2000]
def getEvaluation(results, judgments):
    queryEval = []
    for queryNo in results:
        # doc to relevant score
        relMap = {}
        # number of docs in each relevant score
        relCount = {}
        totalRel = 0
        for j in judgments[queryNo]:
            score = int(j['judgment'])
            relMap[j['cordId']] = score
            relCount[score] = relCount.get(score, 0) + 1
            if score > 0:
                totalRel += 1
        truePositive = 0
        falsePositive = 0
        precisions = []
        reciprocalRank = 0
        cumulativeGain = 0
        discountedCumulativeGain = 0
        idealDCG = 0
        dcgAt = {}
        idcgAt = {}
        ndcgAt = {}
        index = 1
        recallAt = {}
        for score in sorted(relCount.keys(), reverse=True):
            for i in range(relCount[score]):
                idealDCG += (2 ** score - 1) / (math.log2(1 + index))
                if index in ndcg_eval_at:
                    idcgAt[index] = idealDCG
                index += 1
        for (index, doc) in enumerate(results[queryNo]):
            docId = doc['_id']
            score = doc['_score']
            cumulativeGain += relMap.get(docId, 0)
            discountedCumulativeGain += (2 ** (relMap.get(docId, 0)) - 1) / (math.log2(1 + (1 + i)))
            if relMap.get(docId, 0) > 0:
                truePositive += 1
                # recall increase
                precisions.append(truePositive / (truePositive + falsePositive))
                if reciprocalRank == 0:
                    reciprocalRank = 1 / (index+1)
            else:
                falsePositive += 1
            if index + 1 in ndcg_eval_at and (index + 1) in idcgAt:
                dcgAt[index + 1] = discountedCumulativeGain
                ndcgAt[index + 1] = dcgAt[index + 1] / idcgAt[index + 1]
            if index + 1 in recall_eval_at:
                recallAt[index + 1] = truePositive / totalRel
        queryEval.append({
            'AveragePrecision': sum(precisions) / totalRel,
            'ReciprocalRank': reciprocalRank,
            'CG': cumulativeGain,
            'DCG': discountedCumulativeGain,
            'IDCG': idealDCG,
            'nDCG': discountedCumulativeGain / idealDCG,
            'nDCGAt': ndcgAt,
            'recallAt': recallAt
        })
    return {
        'MeanAveragePrecisions': sum(map(lambda e: e['AveragePrecision'], queryEval)) / len(queryEval),
        'MeanReciprocalRank': sum(map(lambda e: e['ReciprocalRank'], queryEval)) / len(queryEval),
        'AverageNDCG': sum(map(lambda e: e['nDCG'], queryEval)) / len(queryEval),
        'AverageNDCGAt': {k: sum(map(lambda e: e['nDCGAt'][k], queryEval)) / len(queryEval) for k in ndcg_eval_at},
        'AverageRecallAt': {k: sum(map(lambda e: e['recallAt'][k], queryEval)) / len(queryEval) for k in recall_eval_at}
    }
            

In [131]:
getEvaluation(results['query'], judgments)

{'MeanAveragePrecisions': 0.1635238316451846,
 'MeanReciprocalRank': 0.8136031746031747,
 'AverageNDCG': 0.38867618286150374,
 'AverageNDCGAt': {3: 0.13364055103725428,
  5: 0.1418463555798589,
  10: 0.1680738131354743,
  15: 0.18464160353165723,
  20: 0.19572503874359487,
  25: 0.20166118601071548,
  30: 0.21199265201748066},
 'AverageRecallAt': {5: 0.007571936093659915,
  10: 0.014347370761125028,
  20: 0.025675868223038946,
  50: 0.05593293689518463,
  100: 0.09603243791764829,
  300: 0.19102147143584872,
  500: 0.24733333654721015,
  1000: 0.33052551322334905,
  2000: 0.4095861748310034}}

In [132]:
getEvaluation(results['question'], judgments)

{'MeanAveragePrecisions': 0.1069352221754944,
 'MeanReciprocalRank': 0.6930461107519932,
 'AverageNDCG': 0.33474171006421577,
 'AverageNDCGAt': {3: 0.09875194087296979,
  5: 0.11636770250055294,
  10: 0.13835908558004478,
  15: 0.1488085799127936,
  20: 0.15100867206756832,
  25: 0.15061045685942806,
  30: 0.15584868010479463},
 'AverageRecallAt': {5: 0.005727560253085045,
  10: 0.011274666322520223,
  20: 0.019611381677786005,
  50: 0.04029976874013658,
  100: 0.06874504517191328,
  300: 0.14134107296584295,
  500: 0.1872360647993138,
  1000: 0.262906206090903,
  2000: 0.3441130401280342}}

In [133]:
getEvaluation(results['narrative'], judgments)

{'MeanAveragePrecisions': 0.09779059095732671,
 'MeanReciprocalRank': 0.6434796152314557,
 'AverageNDCG': 0.31829248511175434,
 'AverageNDCGAt': {3: 0.09286698794380803,
  5: 0.09907587042444932,
  10: 0.12023696099287039,
  15: 0.1276837496201119,
  20: 0.1381839021514124,
  25: 0.1423471423387641,
  30: 0.14730348405985721},
 'AverageRecallAt': {5: 0.005375328395812915,
  10: 0.00957656146663195,
  20: 0.01846469816821065,
  50: 0.03979425169564572,
  100: 0.06755924917514011,
  300: 0.14027205045030436,
  500: 0.18055745790997046,
  1000: 0.249785212170616,
  2000: 0.32925109486394566}}